In [28]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
import os
os.chdir('/content/drive/MyDrive/yolo_newVersion/sattelite_yolo/yolov5')
# path="new_annotation"
# all_data=os.listdir(path)

In [30]:
!ls

benchmarks.py	 data	     LICENSE	  README.zh-CN.md   setup.cfg	    val.py
CITATION.cff	 detect.py   models	  requirements.txt  train.py	    yolov5l.pt
classify	 export.py   __pycache__  runs		    tutorial.ipynb  yolov5s.pt
CONTRIBUTING.md  hubconf.py  README.md	  segment	    utils


## **Inference on image**

In [34]:
import torch
model = torch.hub.load('.', 'custom', path='/content/drive/MyDrive/yolo/sattelite_yolo/yolov5/runs/train/exp3/weights/best.pt', source='local')
# Image
img = '/content/drive/MyDrive/yolo_newVersion/sattelite_yolo/data/test/images/t1.jpg'
# Inference

# .save()

YOLOv5 🚀 v7.0-227-ge4df1ec Python-3.10.12 torch-2.1.0+cu118 CPU

Fusing layers... 
Model summary: 267 layers, 46124433 parameters, 0 gradients, 107.7 GFLOPs
Adding AutoShape... 


In [37]:
%%timeit
results = model(img)


1.46 s ± 378 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## **Inference for vedios**

predicted frames from vedios save in prediction folder of project directory

In [ ]:
import cv2
import numpy as np
# Load the video
video_path = '/content/drive/MyDrive/yolo_newVersion/sattelite_yolo/Maui.MP4'  # Replace with the actual video path
cap = cv2.VideoCapture(video_path)

# Check if the video file is opened successfully
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# Get video frame width, height, and frames per second (fps)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Create a directory to save frames as images
frame_output_dir = '/content/drive/MyDrive/sattelite_yolo/prediction_frames'
os.makedirs(frame_output_dir, exist_ok=True)
frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Resize the input frame
    img = cv2.resize(frame, (640, 640))
    results = model(img)  # Perform inference with YOLOv5

    # Post-process the YOLOv5 results
    results_np = results.pandas().xyxy[0].to_numpy()
    for i in range(len(results_np)):
        x0, y0, x1, y1, conf, cls = results_np[i][:6]
        box = np.array([x0, y0, x1, y1]).astype(np.int32)
        cls_id = int(cls)
        name = model.names[int(cls_id)]
        score = round(float(conf), 2)
        color = (215, 0, 40)
        cv2.rectangle(img, (box[0], box[1]), (box[2], box[3]), color, 2)
        label = f"{name}: {score}"
        label_color = (0, 0, 255)
        cv2.putText(img, label, (box[0], box[1] - 6), cv2.FONT_HERSHEY_SIMPLEX, 0.6, label_color, thickness=2)

    # Save some frames as images
    if frame_count % 50 == 0:  # Save every 50 frames (adjust as needed)
        frame_filename = os.path.join(frame_output_dir, f"frame_{frame_count}.jpg")
        cv2.imwrite(frame_filename, img)
        print(frame_count)
    frame_count += 1

# Release the video capture
cap.release()

0


KeyboardInterrupt: ignored

In [ ]:
import os
from IPython.display import display, Image
# Specify the path to the folder containing saved images
folder_path = '/content/drive/MyDrive/sattelite_yolo/prediction_frames'

# Get a list of all image files in the folder
image_files = [f for f in os.listdir(folder_path) if f.endswith('.jpg')]

# Display each image in the folder
for image_file in image_files:
    image_path = os.path.join(folder_path, image_file)
    display(Image(filename=image_path))


Output hidden; open in https://colab.research.google.com to view.

## **Save the recorded vedio with prediction**

reorded vedio with predicted results are store on path
projectfolder/yolov5/runs/detect/detect/latest_exp/

you can change "projectfolder" and "latest_exp" according to your own

In [ ]:
import os
os.chdir('/content/drive/MyDrive/yolo_newVersion/sattelite_yolo/yolov5')

In [ ]:
!pip install ultralytics

In [ ]:
!python detect.py --weights runs/train/exp3/weights/best.pt --img 640 --conf 0.3 --source /content/drive/MyDrive/yolo_newVersion/sattelite_yolo/data/test/videos

detect: weights=['runs/train/exp3/weights/best.pt'], source=/content/drive/MyDrive/yolo_newVersion/sattelite_yolo/data/test/vedeos, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.3, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-227-ge4df1ec Python-3.10.12 torch-2.1.0+cu118 CPU

Fusing layers... 
Model summary: 267 layers, 46124433 parameters, 0 gradients, 107.7 GFLOPs
Traceback (most recent call last):
  File "/content/drive/MyDrive/yolo_newVersion/sattelite_yolo/yolov5/detect.py", line 285, in <module>
    main(opt)
  File "/content/drive/MyDrive/yolo_newVersion/sattelite_yolo/yolov5/detect.py", line 280, in main
    run(**vars(opt))
  File "/usr/local/lib/python

In [ ]:
import subprocess
import time

video_path = "/content/drive/MyDrive/yolo_newVersion/sattelite_yolo/data/test/videos/istockphoto-1242364184-640_adpp_is.mp4"  # Replace with your video path
frames_to_process = 100  # Number of frames to process for calculating the frame rate

start_time = time.time()
command = f"python detect.py --weights runs/train/exp3/weights/best.pt --img 640 --conf 0.3 --source /content/drive/MyDrive/yolo_newVersion/sattelite_yolo/data/test/videos/istockphoto-1242364184-640_adpp_is.mp4"
process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)
process.wait()
end_time = time.time()

total_time = end_time - start_time
frame_rate = frames_to_process / total_time

print(f"The model is running at approximately {frame_rate} frames per second.")


The model is running at approximately 0.2129619502785272 frames per second.


In [ ]:
import subprocess
import time
import os

def run_inference():
    command = "python detect.py --weights runs/train/exp3/weights/best.pt --img 640 --conf 0.3 --source /content/drive/MyDrive/yolo_newVersion/sattelite_yolo/data/test/videos"
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)

    for line in process.stdout:
        if "Done" in line:  # Assuming "Done" is printed at the end of processing each frame
            print(line.strip())
            print_elapsed_time(line)

def print_elapsed_time(line):
    timestamp = line.split(" ")[0]  # Assuming the timestamp is the first part of the line
    time_format = "%H:%M:%S.%f" if '.' in timestamp else "%H:%M:%S"
    current_time = time.strptime(timestamp, time_format)
    elapsed_time = time.strftime(time_format, time.gmtime(time.mktime(current_time) - time.mktime(start_time)))
    print(f"Elapsed time for the current frame: {elapsed_time}")

start_time = time.localtime()  # Record the starting time

if os.path.exists("/content/drive/MyDrive/yolo_newVersion/sattelite_yolo/data/test/videos"):
    run_inference()
else:
    print("The specified video directory does not exist.")


KeyboardInterrupt: ignored